In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib

from nilearn import plotting

/Users/Work/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# High-Grade Gliomas (HGG)
- Gliomas are tumors.
- HGG are tumors that grow at a high rate.

In [22]:
# HGG Train
# Creating folder path for each folder in directory
folder_contents_path = [('data/MICCAI_BraTS_2019_Data_Training/HGG/' + str(file_name)) for file_name in os.listdir('data/MICCAI_BraTS_2019_Data_Training/HGG') if 'BraTS19' in file_name]

In [30]:
# Getting all image data from HGG Training
hgg_img_data_train = []

for folder_path in folder_contents_path:
    for file in os.listdir(folder_path):
        file_path = folder_path + '/' + str(file)
        
        if '.nii.gz' in file_path:
            img = nib.load(file_path)
            hgg_img_data_train.append(img.get_data())

In [33]:
# To see what my list looks like, delete when complete
# hgg_img_data_train

# Low-Grade Gliomas (LGG)
- Gliomas are tumors.
- LGG are tumors that grow at a slow rate.

In [24]:
# LGG Train
# Creating folder path for each folder in directory
folder_contents_path2 = [('data/MICCAI_BraTS_2019_Data_Training/LGG/' + str(file_name)) for file_name in os.listdir('data/MICCAI_BraTS_2019_Data_Training/LGG') if 'BraTS19' in file_name]


# Getting all image data from LGG Training
lgg_img_data_train = []

for folder_path in folder_contents_path2:
    for file in os.listdir(folder_path):
        file_path = folder_path + '/' + str(file)
        
        if '.nii.gz' in file_path:
            img = nib.load(file_path)
            lgg_img_data_train.append(img.get_data())

In [29]:
# To see what my list looks like, delete when complete
# lgg_img_data_train

In [32]:
# Length of each list

print(f' Length of HGG Train List: {len(hgg_img_data_train)}')
print(f' Length of LGG Train List: {len(lgg_img_data_train)}')

 Length of HGG Train List: 1295
 Length of LGG Train List: 380
